In [1]:
import astropy.units as u

# all the parameters set to fixed will not be fit.  Any other information in the prior dictonary 
# is ignored (e.g., minmax).

# stellar population model
stellar_model = {
    "logA": {  # star formation history SFH
        "name": "bins_histo",
        "x": [6.0, 7.0, 8.0, 9.0, 10.0],  # units are log(years)
        "varnames": ["values"],
        "varinit": [[1e-8, 1e-8, 1e-8, 1e-8, 1e-8]],  # units are M_sun/year
        "prior": {
            "name": "fixed",
            "minmax": [[0.0, 0.0, 0.0, 0.0, 0.0], [1e-3, 1e-3, 1e-3, 1e-3, 1e-3]],
        },
    },
    "M_ini": {  # initial mass function - no parameters yet
        "name": "salpeter",
        "varnames": ["slope"],
        "varinit": [2.35],
        "prior": {
          "name": "fixed",
          "minmax": [[2.0, 3.0]],
        }
    },
    "Z": {
        "name": "flat",
        "prior": {
            "name": "fixed",
        }
    },    
    "distance": {  # no parameters yet
        "name": "absexponential",
        "varnames": ["dist0", "tau", "amp"],
        "varinit": [60.0 * u.kpc, 5. * u.kpc, 1.0],
        "prior": {
            "name": "fixed",
            "minmax": [[50.0, 3.0, 0.9], [70.0, 7.0, 1.1]],
        }
    },
}

# foreground dust cloud
dust_model = {
    "Av": {
        "name": "lognormal",
        "varnames": ["mean", "sigma"],
        "varinit": [4.0, 0.3],
        "prior": {
            "name": "flat",
            "minmax": [[0.005, 5.0], [0.05, 1.0]],
        },
    },
    "Rv": {
        "name": "lognormal",
        "varnames": ["mean", "sigma"],
        "varinit": [3.1, 0.25],
        "prior": {
            "name": "fixed",
            # "name": "flat",
            "minmax": [[2.0, 6.0], [0.05, 1.0]],
        },
    },
    "f_A": {
        "name": "lognormal",
        "varnames": ["mean", "sigma"],
        "varinit": [1.0, 0.25],
        "prior": {
            "name": "fixed",
            # "name": "flat",
            "minmax": [[0.0, 1.0], [0.05, 0.5]],
        },
    }
}

from megabeast.mbmodel import MBModel

mod = MBModel(stellar_model, dust_model)

from megabeast.helpers import read_beast_moddata, get_likelihoods

# BEAST files used by MegaBEAST
beast_base = "scylla_mbtest/scylla_mbtest"
physmodfile = beast_base + "_seds_trim.grid.hd5"
obsmodfile = beast_base + "_noisemodel.grid.hd5"

# get the BEAST physics and observation model info - only that needed for fitting
beast_moddata = read_beast_moddata(physmodfile, obsmodfile, mod.params)

In [3]:
from astropy.io import fits

import numpy as np
from megabeast.mbmodel import MBModel, fit_ensemble, lnprob

basename = "spatial/scylla_mbtest_"

npts = fits.getdata(f"{basename}nstars.fits")
npts_size = npts.shape

av_mean = npts * 0.0
av_sigma = npts * 0.0

# npts_size = [7, 7]
for i in range(npts_size[0]):
    for j in range(npts_size[1]):
        if npts[i, j] > 0:
            mod.physics_model["Av"]["mean"] = 4.0
            mod.physics_model["Av"]["sigma"] = 0.5
            lnpfile = f"{basename}{j}_{i}/{j}_{i}_lnp.hd5"
            star_lnpgriddata = get_likelihoods(lnpfile, beast_moddata)
            sparams = mod.start_params()
            bestparams = fit_ensemble(mod, star_lnpgriddata, beast_moddata, nsteps=100)
            print(bestparams)
            av_mean[i, j] = bestparams[0]
            av_sigma[i, j] = bestparams[1]
            
fits.writeto(f"{basename}av_mean.fits", av_mean, overwrite=True)
fits.writeto(f"{basename}av_sigma.fits", av_sigma, overwrite=True)
            

[1.65505631 0.52402105]
[0.38470841 0.99999724]
[0.19645646 0.94334631]
[0.15319261 0.99999781]
[0.39902607 0.05000117]
[0.49487112 0.99999998]
[0.5729568  0.76738116]
[1.1533983  0.74725242]
[0.51803498 0.99999997]
[1.04182877 0.83556719]
[0.70121832 0.95970647]
[0.37871939 0.89390987]
[0.48555009 0.89666224]
[0.38598004 0.81419635]
[0.28598902 0.92168784]
[0.22004447 0.99999941]
[0.10432017 0.99999983]
[0.9666735  0.93854654]
[0.94290498 0.96321353]
[0.74060594 0.9902731 ]
[0.98971651 0.89077697]
[0.98191583 0.84048988]
[0.4349058  0.90351521]
[0.43997822 0.90958639]
[0.73967016 0.99999998]
[0.68905661 0.99999995]
[0.39035627 0.93523388]
[0.2837524  0.97777289]
[0.42826933 0.76756456]
[0.46322014 0.64649812]
[0.22159457 0.89127882]
[0.26754685 0.76396315]
[2.92569407 0.47467125]
[0.62872508 0.99999985]
[0.69361961 0.99999962]
[0.64409965 0.95731692]
[0.82864119 0.80218217]
[0.36501478 0.99999943]
[0.63446554 0.94093059]
[1.01204355 0.92086201]
[0.447749   0.99999999]
[0.65249556 0.75